In [4]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import h5py
from PIL import Image
from src.benchmarkers import *
from src.benchmarkersV2 import *
from tqdm import tqdm
import os
import numpy as np
from io import BytesIO

np.random.seed(0)

In [6]:
def make_offset(vector):
    return [vector[i] - vector[i - 1] for i in range(1, len(vector))]


def group_objects(objects, cardinality_list):
    result = []
    index = 0
    for count in cardinality_list:
        if count == 0:
            result.append(None)
        if count > 0:  # Only create a group if count > 0
            result.append(objects[index:index+count])
            index += count
        if index > len(objects):
            result.append(None)
    return result

def area(ctx,array):
    format = pc.list_element(array, 4)
    result = []
    for i in range(40):
        if i == 0 :
            mask = pc.equal(format,i)
            result.append(pc.if_else(mask,pc.multiply(pc.list_element(array,2),pc.list_element(array,3)),None))
        elif i == 1 :
            mask = pc.equal(format,i)
            result.append(pc.if_else(mask,pc.multiply(pc.subtract(pc.list_element(array,3),pc.list_element(array,1)),pc.subtract(pc.list_element(array,2),pc.list_element(array,0))),None))
    final_result = 0
    for i, item in enumerate(result):
        if i+1 < len(result):
            final_result = item.fill_null(result[i+1])
    
    return final_result

func_doc = {}
func_doc["summary"] = "calculate area of bboxes based on various formats"
func_doc["description"] = "calculate bbox area"
func_name = "area"
in_types = {"array": pa.list_(pa.float64())}
out_type = pa.float64()
pc.register_scalar_function(area, func_name, func_doc, in_types, out_type)

In [11]:
#table = pq.read_table(r"C:\Users\Cristiano Lavoro\Downloads\ds_10.parquet")
table = pq.read_table(r"C:\Users\rober\onedrive\desktop\projects\benchmarks\imagenette\imagenette2\output\ds.parquet")

In [ ]:
"""""
field1 = table.column("image_feature").chunk(0).values.field("boundingbox_feature").values.field("bbox")
field2 = table.column("image_feature").chunk(0).values.field("boundingbox_feature").values.field("image_1_feature")
field3 = table.column("image_feature").chunk(0).values.field("boundingbox_feature").values.field("label_feature")
new_field = pa.array([73 for i in range(50)])

new_struct_no_cardinality = pa.StructArray.from_arrays(
    [field1,field2, field3, new_field ],  # Both fields
    names=['bbox', 'image_1_feature',"label_feature","area"]  # New child field added
)

cardinality_list = make_offset(table.column("image_feature").chunk(0).values.field("boundingbox_feature").offsets.to_pylist())
new_struct = pa.array(group_objects(new_struct_no_cardinality.to_pylist(), cardinality_list))

# Step 5: Replace the 'age' struct in the 'info' struct
field4 = table.column("image_feature").chunk(0).values.field("image")
field5 = table.column("image_feature").chunk(0).values.field("shape")
#field6 = table.column("image_feature").values.field("boundingbox_feature")
field6 = table.column("image_feature").chunk(0).values.field("text_1_feature")

new_image_feature_column_no_cardinality = pa.StructArray.from_arrays(
    [new_struct,field4,field5,field6],  # Attach the updated 'age' struct
    names=["boundingbox_feature", "image","shape","text_1_feature"]
)

cardinality_list = make_offset(table.column("image_feature").chunk(0).offsets.to_pylist())
new_image_feature_column = pa.array(group_objects(new_image_feature_column_no_cardinality.to_pylist(), cardinality_list))

# Step 6: Replace the original 'info' column in the table with the updated one
new_table = table.set_column(0, 'image_feature', new_image_feature_column)
"""

In [112]:
field1 = table.column("image_feature").chunk(0).values.field("boundingbox_feature").values.field("bbox")
field2 = table.column("image_feature").chunk(0).values.field("boundingbox_feature").values.field("image_1_feature")
field3 = table.column("image_feature").chunk(0).values.field("boundingbox_feature").values.field("label_feature")
new_field = pc.call_function(func_name, [table.column("image_feature").chunk(0).values.field("boundingbox_feature").values.field("bbox")])

new_struct_no_cardinality = pa.StructArray.from_arrays(
    [field1,field2, field3, new_field ],  
    names=['bbox', 'image_1_feature',"label_feature","area"]  
)

cardinality_list = make_offset(table.column("image_feature").chunk(0).values.field("boundingbox_feature").offsets.to_pylist())
new_struct = pa.array(group_objects(new_struct_no_cardinality.to_pylist(), cardinality_list))

field4 = table.column("image_feature").chunk(0).values.field("image")
field5 = table.column("image_feature").chunk(0).values.field("shape")
field6 = table.column("image_feature").chunk(0).values.field("text_1_feature")

new_image_feature_column_no_cardinality = pa.StructArray.from_arrays(
    [new_struct,field4,field5,field6],  
    names=["boundingbox_feature", "image","shape","text_1_feature"]
)

cardinality_list = make_offset(table.column("image_feature").chunk(0).offsets.to_pylist())
new_image_feature_column = pa.array(group_objects(new_image_feature_column_no_cardinality.to_pylist(), cardinality_list))

new_table = table.set_column(0, 'image_feature', new_image_feature_column)

In [31]:
field1 = table.column("image_feature").chunk(0).values.field("image")
#field2 = table.column("image_feature").chunk(0).values.field("class_feature")
field2 = table.column("image_feature").chunk(0).values.field("text_feature")

field_tmp = table.column("image_feature").chunk(0).values.field("class_feature").values
cardinality_list = make_offset(table.column("image_feature").chunk(0).values.field("class_feature").offsets.to_pylist())
field_tmp = group_objects(field_tmp.to_pylist(), cardinality_list)
new_field = [item + [{"label":80}] for item in field_tmp]


new_struct_no_cardinality = pa.StructArray.from_arrays(
    [field1,field2, new_field ],  
    names=['image', 'text_feature',"class_feature"]  
)

cardinality_list = make_offset(table.column("image_feature").chunk(0).offsets.to_pylist())
new_struct = pa.array(group_objects(new_struct_no_cardinality.to_pylist(), cardinality_list))



new_table = table.set_column(0, 'image_feature', new_struct)

In [30]:
len(new_field)

3925

In [124]:
[x.name for x in list(table.column("image_feature").chunk(0).values.field("boundingbox_feature").values.type)]

['bbox', 'image_1_feature', 'label_feature']

In [7]:
def add_struct_field(table : pa.Table,
              feature_list_path: List[str],
              feature_to_manipulate: str,
              new_feature_name : str,
              function_name :str):
    
    obj = None
    to_revert_paths = []

    for indx, feature_name in enumerate(feature_list_path):
        if indx == 0:
            obj = table.column(feature_name).chunk(0)
        else:
            obj = obj.values.field(feature_name)
        
        to_revert_paths.append(obj)

        if indx == len(feature_list_path) - 1:
            obj = obj.values.field(feature_to_manipulate)

    new_field = pc.call_function(function_name, [obj])

    new_struct = None

    for indx,level in enumerate(list(reversed(to_revert_paths))):
        if indx!=0:
            new_field = new_struct
            new_feature_name = list(reversed(feature_list_path))[indx - 1]
        level_fields_values = [new_field]
        level_fields_names = [new_feature_name]
        
        for x in list(level.values.type):
            if not x.name == new_feature_name:
                level_fields_names.append(x.name)
                level_fields_values.append(level.values.field(x.name))    
        new_struct_no_cardinality = pa.StructArray.from_arrays(
            level_fields_values,  
            names=level_fields_names
        )
        cardinality_list = make_offset(level.offsets.to_pylist())
        new_struct = pa.array(group_objects(new_struct_no_cardinality.to_pylist(), cardinality_list))
        
        if indx == len(list(reversed(to_revert_paths))) - 1:
            return table.set_column(table.schema.get_field_index(list(reversed(feature_list_path))[indx]), list(reversed(feature_list_path))[indx], new_struct)
            

In [ ]:
prova = add_struct_field(table,
                 ["image_feature","boundingbox_feature"],
                 "bbox",
                 "area",
                 "area")

prova.column("image_feature").chunk(0).values.field("boundingbox_feature").values.field("area")

In [ ]:
def sum_class_offset(ctx,array):
    return pc.add(array, 1, memory_pool=ctx.memory_pool)

func_doc = {}
func_doc["summary"] = "add an offset to all the classes"
func_doc["description"] = "offset"
func_name = "offset"
in_types = {"array": pa.int64()}
out_type = pa.int64()
pc.register_scalar_function(sum_class_offset, func_name, func_doc, in_types, out_type)

In [9]:
prova2 = add_struct_field(table,
                          ["image_feature","class_feature"],
                          "label",
                          "offset_label",
                          "offset")

In [7]:
def add_feature(table : pa.Table,
              feature_list_path: List[str],
              feature_to_manipulate: str,
              new_feature_name : str,
              function_name :str):

SyntaxError: expected ':' (3001253944.py, line 1)